In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
from sklearn.model_selection import train_test_split

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

from tqdm.auto import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Vocabulary storing how features should be modified at each step
steps = {
    1 : {
        'Dur' : 1, 'SrcBytes' : 1, 'DstBytes' : 1, 'TotPkts' : 1 },
    2 : {
        'Dur' : 2, 'SrcBytes' : 2, 'DstBytes' : 2, 'TotPkts' : 2 },
    3 : {
        'Dur' : 5, 'SrcBytes' : 8, 'DstBytes' : 8, 'TotPkts' : 5 },
    4 : {
        'Dur' : 10, 'SrcBytes' : 16, 'DstBytes' : 16, 'TotPkts' : 10 },
    5 : {
        'Dur' : 15, 'SrcBytes' : 64, 'DstBytes' : 64, 'TotPkts' : 15 },
    6 : {
        'Dur' : 30, 'SrcBytes' : 128, 'DstBytes' : 128, 'TotPkts' : 20 },
    7 : {
        'Dur' : 45, 'SrcBytes' : 256, 'DstBytes' : 256, 'TotPkts' : 30 },
    8 : {
        'Dur' : 60, 'SrcBytes' : 512, 'DstBytes' : 512, 'TotPkts' : 50 },
    9 : {
        'Dur' : 120, 'SrcBytes' : 1024, 'DstBytes' : 1024, 'TotPkts' : 100 }
}

#Groups of altered features
altered_features = {
    '1a' : ['Dur'],
    '1b' : ['SrcBytes'],
    '1c' : ['DstBytes'],
    '1d' : ['TotPkts'],
    '2a' : ['Dur', 'SrcBytes'],
    '2b' : ['Dur', 'DstBytes'],
    '2c' : ['Dur', 'TotPkts'],
    '2d' : ['SrcBytes', 'TotPkts'],
    '2e' : ['SrcBytes', 'DstBytes'],
    '2f' : ['DstBytes', 'TotPkts'],
    '3a' : ['Dur', 'SrcBytes', 'DstBytes'],
    '3b' : ['Dur', 'SrcBytes', 'TotPkts'],
    '3c' : ['Dur', 'DstBytes', 'TotPkts'],
    '3d' : ['SrcBytes','DstBytes','TotPkts'],
    '4a' : ['Dur', 'SrcBytes', 'DstBytes', 'TotPkts']
}

#Function returning the modified dataset according to the list of features to alter
def get_malicious_modified(feature_key, malicious, step):
    for i in altered_features[feature_key]:
        modified = malicious[i] + steps[step][i]
        malicious.loc[:, i] = modified
    return malicious

def correct_derived_features(malicious, ton_iot=False):
    malicious.loc[:, 'TotBytes'] = malicious['SrcBytes'] + malicious['DstBytes']
    if not ton_iot: # ToN_IoT does not have these derived features
        malicious.loc[:, 'BytesPerPkt'] = malicious['TotBytes'] / malicious['TotPkts']
        malicious.loc[:, 'PktsPerSec'] = malicious['TotPkts'] / malicious['Dur']
        max_value = malicious.loc[malicious['PktsPerSec'] != np.inf, 'PktsPerSec'].max()
        malicious.loc[:, 'PktsPerSec'].replace(np.inf, max_value, inplace=True)
        malicious.loc[:, 'RatioOutIn'] = (malicious['DstBytes']) / malicious['SrcBytes']
        max_value = malicious.loc[malicious['RatioOutIn'] != np.inf, 'RatioOutIn'].max()
        malicious.loc[:, 'RatioOutIn'].replace(np.inf, max_value, inplace=True)
    return malicious

def adversarial_feature_attack(test_data, model, ton_iot = False):
    recall_dict = {}
    precision_dict = {}
    f1_dict = {}
    
    pbar = tqdm(range(len(altered_features.keys())*len(steps.keys())))
    for k in altered_features.keys():
        group_recalls = []
        group_precisions = []
        group_f1 = []
        for s in steps.keys():
            data = test_data.copy()
            data[data.Label == 1] = get_malicious_modified(k, data[data.Label == 1], s)
            data[data.Label == 1] = correct_derived_features(data[data.Label == 1], ton_iot)
            
            predictions = model.predict(data.drop(columns=['Label']))
            report = classification_report(data.Label, predictions, digits=3, output_dict=True)
            report = pd.DataFrame(report).transpose()
            
            recall = report.loc['1', 'recall']
            precision = report.loc['1', 'precision']
            f1 = report.loc['1', 'f1-score']
            
            group_recalls.append(recall)
            group_precisions.append(precision)
            group_f1.append(f1)
            
            pbar.write(f"Group {k}, step {s}: {report.loc['1', 'recall']}")
            pbar.update(1)
        
        recall_dict[k] = group_recalls
        precision_dict[k] = group_precisions
        f1_dict[k] = group_f1
        
    recall_df = pd.DataFrame.from_dict(recall_dict)
    precision_df = pd.DataFrame.from_dict(precision_dict)
    f1_df = pd.DataFrame.from_dict(f1_dict)
    
    return recall_df.T, precision_df.T, f1_df.T
            
def test_adversarial_feature_attack():
    data = pd.read_csv("preprocessed_data/murlo_test.csv")
    test_d = data.copy()
    print(f"Original Mal: {test_d.loc[test_d.Label == 1, ['Dur', 'SrcBytes', 'PktsPerSec']].head()}")
    print(f"Original ben: {test_d.loc[test_d.Label == 0, ['Dur', 'SrcBytes', 'PktsPerSec']].head()}")
    
    test_d[test_d.Label == 1] = get_malicious_modified('1a', test_d.loc[test_d.Label == 1], 9)
    test_d[test_d.Label == 1] = correct_derived_features(test_d)
    print(f"Modified Mal: {test_d.loc[test_d.Label == 1, ['Dur', 'SrcBytes', 'PktsPerSec']].head()}")
    print(f"Modified ben: {test_d.loc[test_d.Label == 0, ['Dur', 'SrcBytes', 'PktsPerSec']].head()}")
    print(f"Original TEST DATA: {data.loc[data.Label == 1, ['Dur']].head()}")

In [3]:
import os

In [29]:
if not os.path.exists('results/CTU/scores/rf/'):
    os.mkdir('results/CTU/scores/rf/')

for attack in ['neris', 'rbot', 'virut', 'menti', 'murlo']:

    if not os.path.exists(f'results/CTU/scores/rf/{attack}'):
        os.mkdir(f'results/CTU/scores/rf/{attack}')
    
    train_data = pd.read_csv(f'preprocessed_data/CTU/{attack}_train.csv')
    test_data = pd.read_csv(f'preprocessed_data/CTU/{attack}_test.csv')
    train_data.drop(columns=['Src IP', 'Dst IP'], inplace=True)
    test_data.drop(columns=['Src IP', 'Dst IP'], inplace=True)
    
    model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    model.fit(train_data.drop(columns=['Label']), train_data.Label)
    baseline_pred = model.predict(test_data.drop(columns=['Label']))
    report = classification_report(test_data.Label, baseline_pred, digits=3, output_dict=True)
    report = pd.DataFrame(report).transpose()
    display(report)
    
    report.to_csv(f'results/CTU/scores/rf/{attack}/baseline.csv')
    
    recall, precision, f1 = adversarial_feature_attack(test_data, model)
    display(recall)
    
    recall.to_csv(f'results/CTU/scores/rf/{attack}/feature_attack.csv')

,precision,recall,f1-score,support
0,0.994479,0.996629,0.995553,216539.000000
1,0.963161,0.940939,0.951920,20284.000000
accuracy,0.991859,0.991859,0.991859,0.991859
macro avg,0.978820,0.968784,0.973737,236823.000000
weighted avg,0.991797,0.991859,0.991816,236823.000000


  0%|          | 0/135 [00:00<?, ?it/s]

Group 1a, step 1: 0.8563399723920331
Group 1a, step 2: 0.7394991126010649
Group 1a, step 3: 0.45060145927824885
Group 1a, step 4: 0.8591993689607572
Group 1a, step 5: 0.8142378229146126
Group 1a, step 6: 0.7648392821928613
Group 1a, step 7: 0.7463025044369946
Group 1a, step 8: 0.7436896075724709
Group 1a, step 9: 0.7310195227765727
Group 1b, step 1: 0.4962532044961546
Group 1b, step 2: 0.369453756655492
Group 1b, step 3: 0.23959771248274503
Group 1b, step 4: 0.22737132715440742
Group 1b, step 5: 0.1797968842437389
Group 1b, step 6: 0.14523762571484913
Group 1b, step 7: 0.11945375665549202
Group 1b, step 8: 0.08716229540524552
Group 1b, step 9: 0.06349832380201144
Group 1c, step 1: 0.39232892920528495
Group 1c, step 2: 0.36250246499704203
Group 1c, step 3: 0.2376257148491422
Group 1c, step 4: 0.22623742851508577
Group 1c, step 5: 0.20084795898244923
Group 1c, step 6: 0.17767698678761584
Group 1c, step 7: 0.14789982252021297
Group 1c, step 8: 0.12152435417077499
Group 1c, step 9: 0.09317

,0,1,2,3,4,5,6,7,8
1a,0.856340,0.739499,0.450601,0.859199,0.814238,0.764839,0.746303,0.743690,0.731020
1b,0.496253,0.369454,0.239598,0.227371,0.179797,0.145238,0.119454,0.087162,0.063498
1c,0.392329,0.362502,0.237626,0.226237,0.200848,0.177677,0.147900,0.121524,0.093177
1d,0.348107,0.314632,0.251085,0.225498,0.217610,0.209229,0.203757,0.183938,0.160521
2a,0.435318,0.272777,0.149477,0.137152,0.085437,0.051371,0.037912,0.034362,0.033228
2b,0.317492,0.268389,0.143118,0.125074,0.098403,0.062217,0.050089,0.051075,0.041363
2c,0.290130,0.275932,0.242605,0.221603,0.218645,0.223230,0.213666,0.209870,0.183642
2d,0.245267,0.197939,0.141294,0.117778,0.187389,0.174916,0.164514,0.151942,0.131039
2e,0.378377,0.296884,0.224512,0.208982,0.158598,0.127539,0.102692,0.075675,0.047722
2f,0.310491,0.271495,0.222984,0.202080,0.192073,0.179797,0.184480,0.179895,0.111566


,precision,recall,f1-score,support
0,0.999682,0.999690,0.999686,125958.000000
1,0.994328,0.994184,0.994256,6877.000000
accuracy,0.999405,0.999405,0.999405,0.999405
macro avg,0.997005,0.996937,0.996971,132835.000000
weighted avg,0.999405,0.999405,0.999405,132835.000000


...

,0,1,2,3,4,5,6,7,8
1a,0.990196,0.990196,0.990196,0.990196,0.990196,0.990196,0.990196,0.990196,0.990196
1b,0.995098,0.990196,0.039216,0.039216,0.039216,0.000000,0.000000,0.000000,0.000000
1c,0.995098,0.985294,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216,0.039216
1d,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2a,0.990196,0.990196,0.039216,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2b,0.990196,0.990196,0.039216,0.039216,0.039216,0.039216,0.004902,0.000000,0.000000
2c,0.990196,0.990196,0.990196,0.950980,0.661765,0.950980,0.950980,0.661765,0.661765
2d,1.000000,1.000000,0.656863,0.656863,0.661765,0.000000,0.000000,0.000000,0.000000
2e,0.990196,0.039216,0.039216,0.039216,0.039216,0.000000,0.000000,0.000000,0.000000
2f,1.000000,0.990196,0.421569,0.421569,0.039216,0.039216,0.000000,0.000000,0.000000


,precision,recall,f1-score,support
0,0.999437,0.999437,0.999437,5333.000000
1,0.988722,0.988722,0.988722,266.000000
accuracy,0.998928,0.998928,0.998928,0.998928
macro avg,0.994080,0.994080,0.994080,5599.000000
weighted avg,0.998928,0.998928,0.998928,5599.000000


  0%|          | 0/135 [00:00<?, ?it/s]

Group 1a, step 1: 0.9849624060150376
Group 1a, step 2: 0.9849624060150376
Group 1a, step 3: 0.9849624060150376
Group 1a, step 4: 0.9849624060150376
Group 1a, step 5: 0.9849624060150376
Group 1a, step 6: 0.9849624060150376
Group 1a, step 7: 0.981203007518797
Group 1a, step 8: 0.9699248120300752
Group 1a, step 9: 0.9699248120300752
Group 1b, step 1: 0.9887218045112782
Group 1b, step 2: 0.9887218045112782
Group 1b, step 3: 0.9887218045112782
Group 1b, step 4: 0.9887218045112782
Group 1b, step 5: 0.981203007518797
Group 1b, step 6: 0.9699248120300752
Group 1b, step 7: 0.8721804511278195
Group 1b, step 8: 0.0
Group 1b, step 9: 0.0
Group 1c, step 1: 0.9887218045112782
Group 1c, step 2: 0.9887218045112782
Group 1c, step 3: 0.9849624060150376
Group 1c, step 4: 0.9849624060150376
Group 1c, step 5: 0.9736842105263158
Group 1c, step 6: 0.9736842105263158
Group 1c, step 7: 0.9699248120300752
Group 1c, step 8: 0.9661654135338346
Group 1c, step 9: 0.9661654135338346
Group 1d, step 1: 0.9812030075187

,0,1,2,3,4,5,6,7,8
1a,0.984962,0.984962,0.984962,0.984962,0.984962,0.984962,0.981203,0.969925,0.969925
1b,0.988722,0.988722,0.988722,0.988722,0.981203,0.969925,0.872180,0.000000,0.000000
1c,0.988722,0.988722,0.984962,0.984962,0.973684,0.973684,0.969925,0.966165,0.966165
1d,0.981203,0.917293,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2a,0.984962,0.984962,0.984962,0.977444,0.969925,0.969925,0.000000,0.000000,0.000000
2b,0.984962,0.984962,0.984962,0.977444,0.969925,0.958647,0.000000,0.000000,0.000000
2c,0.981203,0.924812,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2d,0.936090,0.894737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2e,0.988722,0.988722,0.988722,0.984962,0.969925,0.943609,0.000000,0.000000,0.000000
2f,0.981203,0.894737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [4]:
if not os.path.exists('results/ToN_IoT/scores/rf/'):
    os.mkdir('results/ToN_IoT/scores/rf/')

for attack in ['ddos', 'backdoor', 'dos', 'injection', 'password', 'ransomware', 'scanning', 'xss']:

    if not os.path.exists(f'results/ToN_IoT/scores/rf/{attack}'):
        os.mkdir(f'results/ToN_IoT/scores/rf/{attack}')
    
    train_data = pd.read_csv(f'preprocessed_data/ToN_IoT/{attack}_train.csv')
    test_data = pd.read_csv(f'preprocessed_data/ToN_IoT/{attack}_test.csv')
    train_data.drop(columns=['Src IP', 'Dst IP'], inplace=True)
    test_data.drop(columns=['Src IP', 'Dst IP'], inplace=True)
    
    model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
    model.fit(train_data.drop(columns=['Label']), train_data.Label)
    baseline_pred = model.predict(test_data.drop(columns=['Label']))
    report = classification_report(test_data.Label, baseline_pred, digits=3, output_dict=True)
    report = pd.DataFrame(report).transpose()
    display(report)
    
    report.to_csv(f'results/ToN_IoT/scores/rf/{attack}/baseline.csv')
    
    recall, precision, f1 = adversarial_feature_attack(test_data, model)
    display(recall)
    
    recall.to_csv(f'results/ToN_IoT/scores/rf/{attack}/feature_attack.csv')

,precision,recall,f1-score,support
0,0.999796,0.999946,0.999871,73650.000000
1,0.998910,0.995926,0.997416,3682.000000
accuracy,0.999754,0.999754,0.999754,0.999754
macro avg,0.999353,0.997936,0.998644,77332.000000
weighted avg,0.999754,0.999754,0.999754,77332.000000


  0%|          | 0/135 [00:00<?, ?it/s]

Group 1a, step 1: 0.9961977186311787
Group 1a, step 2: 0.9959261271048343
Group 1a, step 3: 0.9956545355784899
Group 1a, step 4: 0.9959261271048343
Group 1a, step 5: 0.996469310157523
Group 1a, step 6: 0.996469310157523
Group 1a, step 7: 0.996469310157523
Group 1a, step 8: 0.9961977186311787
Group 1a, step 9: 0.9940249864204237
Group 1b, step 1: 0.9956545355784899
Group 1b, step 2: 0.9956545355784899
Group 1b, step 3: 0.9953829440521456
Group 1b, step 4: 0.9953829440521456
Group 1b, step 5: 0.9951113525258012
Group 1b, step 6: 0.9937533948940793
Group 1b, step 7: 0.9923954372623575
Group 1b, step 8: 0.989951113525258
Group 1b, step 9: 0.9896795219989136
Group 1c, step 1: 0.9956545355784899
Group 1c, step 2: 0.9956545355784899
Group 1c, step 3: 0.9956545355784899
Group 1c, step 4: 0.9956545355784899
Group 1c, step 5: 0.9951113525258012
Group 1c, step 6: 0.9951113525258012
Group 1c, step 7: 0.9942965779467681
Group 1c, step 8: 0.9926670287887018
Group 1c, step 9: 0.9915806626833242
Group

,0,1,2,3,4,5,6,7,8
1a,0.996198,0.995926,0.995655,0.995926,0.996469,0.996469,0.996469,0.996198,0.994025
1b,0.995655,0.995655,0.995383,0.995383,0.995111,0.993753,0.992395,0.989951,0.989680
1c,0.995655,0.995655,0.995655,0.995655,0.995111,0.995111,0.994297,0.992667,0.991581
1d,0.995926,0.995926,0.995655,0.995383,0.995383,0.995383,0.995383,0.995383,0.995383
2a,0.995926,0.995655,0.995111,0.995383,0.991581,0.989408,0.988593,0.988865,0.960076
2b,0.996198,0.995926,0.995655,0.995655,0.992939,0.989680,0.989680,0.988865,0.989136
2c,0.996198,0.995926,0.995926,0.995926,0.995383,0.994840,0.994025,0.989136,0.986964
2d,0.995383,0.995383,0.995111,0.994840,0.991852,0.989408,0.988050,0.986149,0.950570
2e,0.995383,0.995383,0.995383,0.995383,0.991852,0.989408,0.989408,0.987778,0.951657
2f,0.995655,0.995655,0.995383,0.994840,0.994297,0.992124,0.989680,0.989136,0.987507


,precision,recall,f1-score,support
0,0.999620,0.999740,0.999680,99950.00000
1,0.994785,0.992397,0.993590,4998.00000
accuracy,0.999390,0.999390,0.999390,0.99939
macro avg,0.997203,0.996068,0.996635,104948.00000
weighted avg,0.999390,0.999390,0.999390,104948.00000


...

,0,1,2,3,4,5,6,7,8
1a,0.996705,0.995881,0.995881,0.995881,0.995881,0.995881,0.995881,0.995675,0.995675
1b,0.997529,0.997529,0.996911,0.996911,0.996293,0.995470,0.995470,0.994852,0.994852
1c,0.997529,0.997529,0.997529,0.997529,0.997323,0.997323,0.997117,0.996911,0.996911
1d,0.997529,0.997323,0.996911,0.996705,0.996499,0.996499,0.996499,0.996499,0.996499
2a,0.996705,0.995675,0.995675,0.995470,0.995058,0.991557,0.991557,0.991351,0.991145
2b,0.996705,0.995881,0.995881,0.995675,0.995470,0.995264,0.995058,0.995058,0.995058
2c,0.996499,0.988056,0.327842,0.327842,0.327842,0.327842,0.327842,0.327636,0.364498
2d,0.996911,0.996499,0.995264,0.995058,0.994852,0.912479,0.912273,0.911862,0.934926
2e,0.997529,0.997529,0.996911,0.996293,0.995470,0.995058,0.994852,0.994646,0.994234
2f,0.996911,0.996499,0.996499,0.996087,0.995058,0.994852,0.994852,0.960873,0.960873


,precision,recall,f1-score,support
0,0.998383,0.995935,0.997158,93000.000000
1,0.922509,0.967742,0.944584,4650.000000
accuracy,0.994593,0.994593,0.994593,0.994593
macro avg,0.960446,0.981839,0.970871,97650.000000
weighted avg,0.994770,0.994593,0.994654,97650.000000


  0%|          | 0/135 [00:00<?, ?it/s]

Group 1a, step 1: 0.9759139784946237
Group 1a, step 2: 0.9701075268817204
Group 1a, step 3: 0.9696774193548388
Group 1a, step 4: 0.9696774193548388
Group 1a, step 5: 0.9696774193548388
Group 1a, step 6: 0.9705376344086022
Group 1a, step 7: 0.970752688172043
Group 1a, step 8: 0.9709677419354839
Group 1a, step 9: 0.9765591397849462
Group 1b, step 1: 0.9664516129032258
Group 1b, step 2: 0.9664516129032258
Group 1b, step 3: 0.9653763440860215
Group 1b, step 4: 0.9311827956989247
Group 1b, step 5: 0.930752688172043
Group 1b, step 6: 0.9305376344086022
Group 1b, step 7: 0.9653763440860215
Group 1b, step 8: 0.9662365591397849
Group 1b, step 9: 0.9688172043010753
Group 1c, step 1: 0.9673118279569892
Group 1c, step 2: 0.9673118279569892
Group 1c, step 3: 0.9666666666666667
Group 1c, step 4: 0.9666666666666667
Group 1c, step 5: 0.9662365591397849
Group 1c, step 6: 0.9662365591397849
Group 1c, step 7: 0.9666666666666667
Group 1c, step 8: 0.9666666666666667
Group 1c, step 9: 0.9673118279569892
Gro

,0,1,2,3,4,5,6,7,8
1a,0.975914,0.970108,0.969677,0.969677,0.969677,0.970538,0.970753,0.970968,0.976559
1b,0.966452,0.966452,0.965376,0.931183,0.930753,0.930538,0.965376,0.966237,0.968817
1c,0.967312,0.967312,0.966667,0.966667,0.966237,0.966237,0.966667,0.966667,0.967312
1d,0.967742,0.967097,0.931613,0.926452,0.923226,0.920215,0.918495,0.912903,0.911828
2a,0.974624,0.967527,0.933118,0.932903,0.932903,0.933118,0.967527,0.968817,0.977634
2b,0.975484,0.969462,0.934194,0.933548,0.933548,0.933978,0.935054,0.935699,0.970968
2c,0.970753,0.968387,0.933333,0.928387,0.673978,0.670108,0.669892,0.680430,0.685376
2d,0.966237,0.963441,0.912473,0.889892,0.883656,0.876129,0.889462,0.900000,0.911613
2e,0.966452,0.966237,0.931398,0.930753,0.930323,0.930323,0.965591,0.967312,0.968817
2f,0.967097,0.966022,0.926022,0.913333,0.907957,0.898495,0.900430,0.898495,0.894194
